In [ ]:
import tensorflow as tf
from PIL import Image
import numpy as np
import cv2

In [ ]:
tf.enable_eager_execution()

In [ ]:
from superglue.keypoint_extractors import SuperPointExtractor, Keypoints
from superglue.datasets import create_training_image_pair_sampler_fn

In [ ]:
extractor = SuperPointExtractor("/home/mkk/Downloads/superglue/model/sp_v5/")

In [ ]:
image_size=(512, 512)

sample_fn = create_training_image_pair_sampler_fn(
    extractor=extractor,
    image_size=image_size,
    reprojection_threshold = 5, 
    num_matches=512,
)

In [ ]:
from pathlib import Path
images_paths = list(Path("/home/mkk/Downloads/superglue/images/").glob("*.jpg") )
image = Image.open(images_paths[0]).resize(image_size)
image_np = np.array(image)

In [ ]:
input_image = np.array(image.resize(image_size))
transformed_image, (features, labels) = sample_fn(image_np)

In [ ]:
fa, ka, fb, kb = features

In [ ]:
cv_ka = Keypoints.convert_numpy_keypoints_to_cv(ka.numpy() * np.array([image_size]))
cv_kb = Keypoints.convert_numpy_keypoints_to_cv(kb.numpy() * np.array([image_size]))

In [ ]:
np.where(np.diff(labels[:, -1]) == 1)[0]

In [ ]:
img_with_keypoints = cv2.drawKeypoints(
    input_image, 
    cv_ka, 
    outImage=None, color=(0, 0, 255), 
    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)

In [ ]:
Image.fromarray(img_with_keypoints)

In [ ]:
img_with_keypoints = cv2.drawKeypoints(
    transformed_image.numpy().astype("uint8"), 
    cv_kb, 
    outImage=None, color=(0, 0, 255), 
    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)

In [ ]:
Image.fromarray(img_with_keypoints)

In [ ]:
np.where(np.diff(labels[:, -1]) == 1)[0][0]

In [ ]:
start = 0
end = 10
img_with_keypoints = cv2.drawMatches(
    input_image, cv_ka[start:end],
    transformed_image.numpy().astype("uint8"), cv_kb[start:end],
    [cv2.DMatch(_queryIdx=i, _trainIdx=i, _distance=0) for i in range(end - start)], outImg=None
)

In [ ]:
Image.fromarray(img_with_keypoints)

# Dataset generator

In [ ]:
from superglue.datasets import superpoint_image_pair_generator

In [ ]:
train_generator = superpoint_image_pair_generator(
    images_dir="/home/mkk/Downloads/superglue/images/",
    model_dir="/home/mkk/Downloads/superglue/model/sp_v5/",
    image_size=image_size
)

In [ ]:
features, labels = next(train_generator)

In [ ]:
import tensorflow as tf

In [ ]:
dataset = tf.data.Dataset.from_generator(lambda: train_generator, ((tf.float32, tf.float32, tf.float32, tf.float32), tf.float32))
dataset = dataset.batch(8).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
for features, labels in dataset:
    features
    break